<a href="https://colab.research.google.com/github/Kawarini1/Keattisak/blob/main/data_analyz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install matplotlib
!pip install seaborn
!pip install joblib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold ,cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
# อ่านข้อมูล
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/colab/train set.csv')

In [ ]:

# ข้อมูลที่ใช้สำหรับการ train
X = data.drop("AQI", axis = 1)
y = data["AQI"]
print(X)

# แสดงประเภทของข้อมูล
print(data.dtypes)

# แสดงคอลัมน์ทั้งหมดใน DataFrame
print("Columns in the DataFrame:", data.columns)

# ตรวจสอบว่าคอลัมน์ 'AQI' มีอยู่จริงหรือไม่
if 'AQI' not in data.columns:
    raise ValueError("Column 'AQI ' not found in the DataFrame")

            PM10       PM2_5         NO2        SO2  Temperature   Humidity
0     295.853039   13.038560    6.639263  66.161150     5.150335  84.424344
1     246.254703    9.984497   16.318326  90.499523     1.543378  46.851415
2      84.443191   23.111340   96.317811  17.875850     1.169483  17.806977
3      21.020609   14.273403   81.234403  48.323616    21.925276  99.473373
4      16.987667  152.111623  121.235461  90.866167     9.217517  24.906837
...          ...         ...         ...        ...          ...        ...
5806   11.246387  197.984629  158.643107  17.743678    37.359323  61.707640
5807  275.340762   55.774170  132.336871  29.334724    34.532542  21.528555
5808   41.892699  184.708551   82.105823  68.334578    22.975564  92.725625
5809  165.533785  199.177255  100.796385  87.586488    36.090620  25.836286
5810   82.216262  119.968244  193.440691  79.849177    18.239090  77.096194

[5811 rows x 6 columns]
PM10           float64
PM2_5          float64
NO2            fl

In [ ]:

# เลือกฟีเจอร์ที่ดีที่สุด
X_new = SelectKBest(score_func=f_classif, k=10).fit_transform(X, y)
print(X_new)
# แบ่งข้อมูลเป็นชุดฝึกและชุดทดสอบ
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)

# กำหนดพารามิเตอร์สำหรับ Random Search
param_dist = {
    'n_estimators': np.arange(50, 200, 10),
    'max_depth': [None] + list(np.arange(5, 20, 2)),
    'min_samples_split': np.arange(2, 10, 2),
    'min_samples_leaf': np.arange(1, 5, 1)}

/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:783: UserWarning: k=10 is greater than n_features=6. All the features will be returned.
  warnings.warn(


[[295.8530392   13.03856044   6.63926301  66.16114965   5.15033504
   84.42434365]
 [246.2547028    9.98449713  16.31832608  90.4995226    1.54337829
   46.85141478]
 [ 84.44319074  23.11133977  96.31781102  17.87585035   1.16948342
   17.80697722]
 ...
 [ 41.89269906 184.7085514   82.10582279  68.33457751  22.97556362
   92.72562474]
 [165.5337851  199.1772552  100.7963851   87.58648826  36.09062031
   25.83628643]
 [ 82.21626223 119.9682442  193.4406912   79.84917706  18.23908975
   77.09619361]]


/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:108: RuntimeWarning: invalid value encountered in divide
  msw = sswn / float(dfwn)


In [ ]:
# ตรวจสอบจำนวนสมาชิกในแต่ละคลาส
unique, counts = np.unique(y, return_counts=True)
class_counts = dict(zip(unique, counts))

# ตรวจสอบให้แน่ใจว่า n_splits น้อยกว่าหรือเท่ากับจำนวนสมาชิกในคลาสที่มีจำนวนน้อยที่สุด
min_class_count = min(class_counts.values())

# กำหนดจำนวนการแบ่งข้อมูล
n_splits = 5

# ตรวจสอบให้แน่ใจว่า n_splits อย่างน้อย 2
if n_splits < 2:
    raise ValueError(f"n_splits={n_splits} cannot be less than 2.")

# การใช้ stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=n_splits)
print(skf)

StratifiedKFold(n_splits=5, random_state=None, shuffle=False)


In [ ]:
# สร้างโมเดล RandomForestClassifier
clf = RandomForestRegressor(random_state=42)

# สร้าง Random Search
random_search = RandomizedSearchCV(estimator=clf, param_distributions=param_dist, n_iter=100, cv=5, n_jobs=-1, random_state=42, verbose=2)

# ฝึกโมเดลด้วย Random Search
random_search.fit(X_train, y_train)

# แสดงพารามิเตอร์ที่ดีที่สุด
print("Best parameters found: ", random_search.best_params_)

# สร้างโมเดลที่ดีที่สุด
best_clf = random_search.best_estimator_

# สร้างโมเดลแบบ Ensemble โดยใช้ Voting
ensemble_clf = VotingClassifier(estimators=[
    ('rf', best_clf),
    ('rf2', RandomForestClassifier(n_estimators=best_clf.n_estimators, max_depth=best_clf.max_depth, min_samples_split=best_clf.min_samples_split, min_samples_leaf=best_clf.min_samples_leaf, random_state=43))
], voting='soft')

# ฝึกโมเดล Ensemble
ensemble_clf.fit(X_train, y_train)

# ใช้ Cross-Validation เพื่อประเมินผลโมเดล
cv_scores = cross_val_score(ensemble_clf, X_train, y_train, cv=5)

# แสดงผลลัพธ์ Cross-Validation
print("Cross-Validation Accuracy Scores: ", cv_scores)
print("Mean Accuracy: ", cv_scores.mean())
print("Standard Deviation: ", cv_scores.std())

# ทำพยากรณ์ด้วยโมเดล Ensemble
y_pred = ensemble_clf.predict(X_test)

# วัดความแม่นยำ
accuracy = accuracy_score(y_test, y_pred)
print("Ensemble Model Accuracy on Test Set: ", accuracy)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


In [ ]:
import joblib
joblib.dump(ensemble_clf, 'ensemble_clf_model2.pkl')

In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
   # สร้าง LabelEncoder
encoder = LabelEncoder()

load_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/colab/Test set.csv")

    # สมมติว่า new_data เป็น DataFrame ของข้อมูลใหม่
new_data = pd.DataFrame(load_data)

   # แปลงค่าสตริงในคอลัมน์ "aqi" เป็นตัวเลข
target_data = encoder.fit_transform(load_data["aqi"])

def get_aqi_category(aqi):
    if aqi <= 50:
        return "Good"
    elif aqi <= 100:
        return "Moderate"
    elif aqi <= 150:
        return "Unhealthy for Sensitive Groups"
    elif aqi <= 200:
        return "Unhealthy"
    elif aqi <= 300:
        return "Very Unhealthy"

   # เลือก Feature ที่ดีที่สุด
X_new_data = SelectKBest(score_func=f_classif, k=10).fit_transform(new_data, "aqi")


In [ ]:
# ทำนายผลลัพธ์
predictions = ensemble_clf.predict(X_test)
print(predictions)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# สร้าง RandomForestClassifier เพื่อหาความสำคัญของฟีเจอร์
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X, y)

# ความสำคัญของฟีเจอร์
feature_importance = rf.feature_importances_

# สร้างกราฟแสดงความสำคัญของฟีเจอร์
plt.figure(figsize=(10, 6))
plt.barh(X.columns, feature_importance, color='skyblue')
plt.xlabel('Feature Importance')
plt.title('Feature Importance from RandomForest')
plt.show()

# หากต้องการสร้างกราฟการกระจาย (Pairplot) ด้วย Seaborn
sns.pairplot(data, hue="AQI_level", diag_kind="kde", markers=["o", "s", "D", "v", "^"])
plt.suptitle("Pairplot of Features", y=1.02)
plt.show()

# หากต้องการสร้างกราฟการกระจายข้อมูลระหว่าง Features
sns.heatmap(X.corr(), annot=True, cmap="coolwarm", linewidths=0.5)
plt.title("Correlation Heatmap of Features")
plt.show()